# Corrected Futures Data Visualization

Fixed version that works with the actual data structure in the database.

In [ ]:
import sys
import os
import pandas as pd
import numpy as np
import pyodbc
from datetime import datetime, timedelta
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
from matplotlib.gridspec import GridSpec
import seaborn as sns
import warnings

# Add project root to Python path
project_root = os.path.dirname(os.path.dirname(os.path.abspath('__file__')))
sys.path.insert(0, project_root)

from config.database_config import get_connection_string

warnings.filterwarnings('ignore')

# Style settings
plt.style.use('seaborn-v0_8-whitegrid')
plt.rcParams['font.family'] = 'DejaVu Sans'
plt.rcParams['font.size'] = 11
plt.rcParams['axes.labelsize'] = 12
plt.rcParams['axes.titlesize'] = 14
plt.rcParams['figure.titlesize'] = 16
plt.rcParams['figure.dpi'] = 100
plt.rcParams['savefig.dpi'] = 150
plt.rcParams['axes.grid'] = True
plt.rcParams['grid.alpha'] = 0.3

# Color palette
COLORS = {
    'primary': '#2E86AB',
    'secondary': '#A23B72',
    'accent': '#F18F01',
    'success': '#C73E1D',
    'dark': '#2D3436',
    'light': '#F7F7F7'
}

# Gradient colors
gradient_colors = ['#667eea', '#764ba2', '#f093fb', '#f5576c', '#4facfe', '#00f2fe']

print("Libraries imported successfully")

## Data Retrieval (Corrected)

In [ ]:
def get_futures_data(conn, days=90):
    """Retrieve futures data - corrected version"""
    query = f"""
    SELECT 
        p.TradeDate,
        m.MetalCode,
        m.ExchangeCode,
        t.TenorTypeName,
        p.SettlementPrice,
        p.Volume,
        p.OpenInterest,
        CASE 
            WHEN t.TenorTypeName LIKE 'Generic 1st%' THEN 1
            WHEN t.TenorTypeName LIKE 'Generic 2nd%' THEN 2
            WHEN t.TenorTypeName LIKE 'Generic 3rd%' THEN 3
            WHEN t.TenorTypeName LIKE 'Generic 4th%' THEN 4
            WHEN t.TenorTypeName LIKE 'Generic 5th%' THEN 5
            WHEN t.TenorTypeName LIKE 'Generic 6th%' THEN 6
            WHEN t.TenorTypeName LIKE 'Generic 7th%' THEN 7
            WHEN t.TenorTypeName LIKE 'Generic 8th%' THEN 8
            WHEN t.TenorTypeName LIKE 'Generic 9th%' THEN 9
            WHEN t.TenorTypeName LIKE 'Generic 10th%' THEN 10
            WHEN t.TenorTypeName LIKE 'Generic 11th%' THEN 11
            WHEN t.TenorTypeName LIKE 'Generic 12th%' THEN 12
            ELSE 0
        END as TenorNumber
    FROM T_CommodityPrice p
    INNER JOIN M_Metal m ON p.MetalID = m.MetalID
    INNER JOIN M_TenorType t ON p.TenorTypeID = t.TenorTypeID
    WHERE 
        t.TenorTypeName LIKE 'Generic%Future%'
        AND p.TradeDate >= DATEADD(day, -{days}, GETDATE())
        AND p.SettlementPrice IS NOT NULL
        AND m.MetalCode = 'COPPER'
    ORDER BY p.TradeDate DESC, t.TenorTypeName
    """
    
    print(f"Executing query for last {days} days...")
    
    with warnings.catch_warnings():
        warnings.filterwarnings("ignore", message="pandas only supports SQLAlchemy")
        df = pd.read_sql(query, conn)
    
    df['TradeDate'] = pd.to_datetime(df['TradeDate'])
    return df

# Connect and get data
conn = pyodbc.connect(get_connection_string())
print("Connected to database")

futures_df = get_futures_data(conn, days=90)
print(f"\nRetrieved {len(futures_df):,} records")

if len(futures_df) > 0:
    print(f"Date range: {futures_df['TradeDate'].min().strftime('%Y-%m-%d')} to {futures_df['TradeDate'].max().strftime('%Y-%m-%d')}")
    print(f"Tenor numbers found: {sorted(futures_df[futures_df['TenorNumber'] > 0]['TenorNumber'].unique())}")
    print("\nSample data:")
    print(futures_df.head())
else:
    print("No data found!")

## 1. Copper Futures Curve - Latest Trading Day

In [ ]:
if len(futures_df) > 0:
    # Get latest trading day
    latest_date = futures_df['TradeDate'].max()
    latest_data = futures_df[futures_df['TradeDate'] == latest_date].copy()
    
    # Filter valid tenor numbers and sort
    latest_data = latest_data[latest_data['TenorNumber'] > 0].sort_values('TenorNumber')
    
    if len(latest_data) > 0:
        # Create plot
        fig, ax = plt.subplots(figsize=(14, 8))
        
        # Plot futures curve
        x = latest_data['TenorNumber']
        y = latest_data['SettlementPrice']
        
        # Fill area under curve
        ax.fill_between(x, y, alpha=0.3, color=COLORS['primary'], label='Price Range')
        
        # Main line
        ax.plot(x, y, color=COLORS['primary'], linewidth=4, marker='o', 
                markersize=10, markerfacecolor='white', markeredgewidth=3, 
                markeredgecolor=COLORS['primary'], label='Settlement Price')
        
        # Add value labels
        for tenor, price in zip(x, y):
            ax.annotate(f'${price:,.0f}', (tenor, price), 
                       textcoords="offset points", xytext=(0,15), 
                       ha='center', fontsize=10, weight='bold',
                       bbox=dict(boxstyle='round,pad=0.3', facecolor='white', 
                                edgecolor=COLORS['primary'], alpha=0.8))
        
        # Styling
        ax.set_title(f'Copper Futures Curve - {latest_date.strftime("%Y-%m-%d")}',
                    fontsize=18, weight='bold', pad=25)
        ax.set_xlabel('Contract Month', fontsize=14, weight='bold')
        ax.set_ylabel('Price (USD/tonne)', fontsize=14, weight='bold')
        ax.set_xticks(x)
        ax.set_xticklabels([f'M{i}' for i in x])
        ax.grid(True, alpha=0.3)
        ax.legend(loc='best', fontsize=12)
        
        # Add some statistics
        plt.figtext(0.02, 0.02, 
                   f'Front Month: ${y.iloc[0]:,.0f} | Average: ${y.mean():,.0f} | Range: ${y.max()-y.min():,.0f}',
                   fontsize=10, style='italic')
        
        plt.tight_layout()
        plt.show()
        
        # Calculate and display term structure
        if len(latest_data) >= 3:
            m1_price = latest_data[latest_data['TenorNumber']==1]['SettlementPrice'].iloc[0]
            m3_price = latest_data[latest_data['TenorNumber']==3]['SettlementPrice'].iloc[0]
            spread = m3_price - m1_price
            structure = "Contango" if spread > 0 else "Backwardation"
            
            print(f"\nTerm Structure Analysis:")
            print(f"M1 Price: ${m1_price:,.2f}")
            print(f"M3 Price: ${m3_price:,.2f}")
            print(f"M1-M3 Spread: ${spread:,.2f}")
            print(f"Market Structure: {structure}")
    else:
        print("No valid futures data for latest date")
else:
    print("No futures data available")

## 2. Price Evolution Heatmap

In [ ]:
if len(futures_df) > 0:
    # Get last 30 trading days
    recent_dates = sorted(futures_df['TradeDate'].unique(), reverse=True)[:30]
    recent_data = futures_df[futures_df['TradeDate'].isin(recent_dates)].copy()
    
    # Filter valid tenor numbers
    recent_data = recent_data[recent_data['TenorNumber'] > 0]
    
    # Create pivot table
    pivot_data = recent_data.pivot_table(
        values='SettlementPrice',
        index='TradeDate',
        columns='TenorNumber',
        aggfunc='mean'
    ).sort_index(ascending=True)
    
    if not pivot_data.empty and len(pivot_data.columns) > 1:
        # Create heatmap
        fig, ax = plt.subplots(figsize=(14, 10))
        
        # Use seaborn heatmap with custom colormap
        sns.heatmap(pivot_data, 
                   cmap='RdYlBu_r',
                   center=pivot_data.mean().mean(),
                   cbar_kws={'label': 'Price (USD/tonne)', 'shrink': 0.8},
                   linewidths=0.5,
                   ax=ax,
                   robust=True)
        
        # Customize labels
        ax.set_xticklabels([f'M{int(col)}' for col in pivot_data.columns])
        ax.set_yticklabels([date.strftime('%m/%d') for date in pivot_data.index], rotation=0)
        
        ax.set_title('Copper Futures Price Evolution - Last 30 Trading Days', 
                    fontsize=16, weight='bold', pad=20)
        ax.set_xlabel('Contract Month', fontsize=14, weight='bold')
        ax.set_ylabel('Trading Date', fontsize=14, weight='bold')
        
        plt.tight_layout()
        plt.show()
        
        # Price change analysis
        price_changes = pivot_data.pct_change().dropna()
        avg_volatility = price_changes.std() * np.sqrt(252)  # Annualized
        
        print("\nVolatility Analysis (Annualized):")
        for tenor in avg_volatility.index:
            print(f"M{tenor}: {avg_volatility[tenor]:.1%}")
    else:
        print("Not enough data for heatmap")
else:
    print("No data for heatmap")

## 3. Calendar Spreads Analysis

In [ ]:
if len(futures_df) > 0:
    # Calculate calendar spreads for the latest date
    latest_date = futures_df['TradeDate'].max()
    latest_data = futures_df[futures_df['TradeDate'] == latest_date].copy()
    latest_data = latest_data[latest_data['TenorNumber'] > 0].sort_values('TenorNumber')
    
    if len(latest_data) >= 2:
        # Calculate spreads
        spreads = []
        spread_labels = []
        prices = latest_data.set_index('TenorNumber')['SettlementPrice']
        
        for i in range(1, len(prices)):
            spread = prices.iloc[i] - prices.iloc[i-1]
            spreads.append(spread)
            spread_labels.append(f'M{prices.index[i-1]}-M{prices.index[i]}')
        
        # Create visualization
        fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(14, 10), 
                                      gridspec_kw={'height_ratios': [2, 1]})
        
        # Top: Futures curve with spread indicators
        ax1.plot(prices.index, prices.values, 'o-', color=COLORS['primary'], 
                linewidth=3, markersize=8, label='Settlement Prices')
        
        # Color code the line segments based on spread direction
        for i, (spread, label) in enumerate(zip(spreads, spread_labels)):
            color = COLORS['success'] if spread > 0 else COLORS['secondary']
            x_start, x_end = prices.index[i], prices.index[i+1]
            y_start, y_end = prices.iloc[i], prices.iloc[i+1]
            ax1.plot([x_start, x_end], [y_start, y_end], 
                    color=color, linewidth=5, alpha=0.7)
        
        ax1.set_title(f'Futures Curve with Calendar Spreads - {latest_date.strftime("%Y-%m-%d")}',
                     fontsize=16, weight='bold')
        ax1.set_xlabel('Contract Month', fontsize=12)
        ax1.set_ylabel('Price (USD/tonne)', fontsize=12)
        ax1.set_xticks(prices.index)
        ax1.set_xticklabels([f'M{i}' for i in prices.index])
        ax1.grid(True, alpha=0.3)
        ax1.legend()
        
        # Bottom: Calendar spreads bar chart
        colors = [COLORS['success'] if s > 0 else COLORS['secondary'] for s in spreads]
        bars = ax2.bar(range(len(spreads)), spreads, color=colors, alpha=0.8)
        
        # Add value labels on bars
        for bar, val in zip(bars, spreads):
            height = bar.get_height()
            ax2.text(bar.get_x() + bar.get_width()/2., height + (2 if height > 0 else -8),
                    f'${val:.1f}', ha='center', va='bottom' if height > 0 else 'top',
                    fontsize=10, weight='bold')
        
        ax2.set_title('Calendar Spreads (Adjacent Months)', fontsize=14, weight='bold')
        ax2.set_xticks(range(len(spreads)))
        ax2.set_xticklabels(spread_labels, rotation=45, ha='right')
        ax2.set_ylabel('Spread (USD/tonne)', fontsize=12)
        ax2.axhline(y=0, color='black', linestyle='--', alpha=0.5)
        ax2.grid(True, alpha=0.3, axis='y')
        
        plt.tight_layout()
        plt.show()
        
        # Spread statistics
        avg_spread = np.mean(spreads)
        max_spread = max(spreads)
        min_spread = min(spreads)
        
        print(f"\nCalendar Spread Analysis:")
        print(f"Average spread: ${avg_spread:.2f}")
        print(f"Maximum contango: ${max_spread:.2f} ({spread_labels[spreads.index(max_spread)]})")
        print(f"Maximum backwardation: ${min_spread:.2f} ({spread_labels[spreads.index(min_spread)]})")
        
        overall_structure = "Contango" if avg_spread > 0 else "Backwardation"
        print(f"Overall market structure: {overall_structure}")
    else:
        print("Not enough data for spread analysis")
else:
    print("No data for spread analysis")

## 4. Volume and Open Interest Analysis

In [ ]:
if len(futures_df) > 0:
    # Calculate average volume and OI by tenor
    liquidity_data = futures_df[futures_df['TenorNumber'] > 0].groupby('TenorNumber').agg({
        'Volume': ['mean', 'std'],
        'OpenInterest': ['mean', 'std']
    }).round(0)
    
    if not liquidity_data.empty:
        fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(16, 7))
        
        # Volume chart
        tenors = liquidity_data.index
        volume_mean = liquidity_data[('Volume', 'mean')]
        volume_std = liquidity_data[('Volume', 'std')].fillna(0)
        
        bars1 = ax1.bar(tenors, volume_mean, yerr=volume_std, 
                        color=[gradient_colors[i % len(gradient_colors)] for i in range(len(tenors))],
                        alpha=0.8, capsize=5, error_kw={'linewidth': 2, 'ecolor': 'gray'})
        
        # Add value labels
        for i, (tenor, vol) in enumerate(zip(tenors, volume_mean)):
            if vol > 0:  # Only show non-zero values
                ax1.text(tenor, vol + volume_std.iloc[i] + max(volume_mean) * 0.02, 
                        f'{vol:,.0f}', ha='center', va='bottom', 
                        fontsize=9, weight='bold')
        
        ax1.set_title('Average Daily Volume by Contract Month', fontsize=14, weight='bold')
        ax1.set_xlabel('Contract Month', fontsize=12)
        ax1.set_ylabel('Volume (contracts)', fontsize=12)
        ax1.set_xticks(tenors)
        ax1.set_xticklabels([f'M{i}' for i in tenors])
        ax1.grid(True, alpha=0.3, axis='y')
        
        # Open Interest chart
        oi_mean = liquidity_data[('OpenInterest', 'mean')]
        oi_std = liquidity_data[('OpenInterest', 'std')].fillna(0)
        
        bars2 = ax2.bar(tenors, oi_mean, yerr=oi_std,
                        color=[gradient_colors[(i+3) % len(gradient_colors)] for i in range(len(tenors))],
                        alpha=0.8, capsize=5, error_kw={'linewidth': 2, 'ecolor': 'gray'})
        
        # Add value labels
        for i, (tenor, oi) in enumerate(zip(tenors, oi_mean)):
            if oi > 0:  # Only show non-zero values
                ax2.text(tenor, oi + oi_std.iloc[i] + max(oi_mean) * 0.02, 
                        f'{oi:,.0f}', ha='center', va='bottom', 
                        fontsize=9, weight='bold')
        
        ax2.set_title('Average Open Interest by Contract Month', fontsize=14, weight='bold')
        ax2.set_xlabel('Contract Month', fontsize=12)
        ax2.set_ylabel('Open Interest (contracts)', fontsize=12)
        ax2.set_xticks(tenors)
        ax2.set_xticklabels([f'M{i}' for i in tenors])
        ax2.grid(True, alpha=0.3, axis='y')
        
        plt.suptitle('Copper Futures Liquidity Analysis', fontsize=16, weight='bold', y=1.02)
        plt.tight_layout()
        plt.show()
        
        # Liquidity concentration analysis
        total_volume = volume_mean.sum()
        total_oi = oi_mean.sum()
        
        if len(volume_mean) >= 3 and total_volume > 0 and total_oi > 0:
            front_3_volume = volume_mean[:3].sum() / total_volume * 100
            front_3_oi = oi_mean[:3].sum() / total_oi * 100
            
            print(f"\nLiquidity Concentration Analysis:")
            print(f"Front 3 months volume share: {front_3_volume:.1f}%")
            print(f"Front 3 months open interest share: {front_3_oi:.1f}%")
            
            most_liquid = volume_mean.idxmax()
            print(f"Most liquid contract: M{most_liquid} ({volume_mean[most_liquid]:,.0f} contracts/day)")
    else:
        print("No liquidity data available")
else:
    print("No data for liquidity analysis")

## Close Database Connection

In [ ]:
conn.close()
print("Database connection closed successfully")